# Set up Overmind

Set api keys as env variables. Alternatively they can be passed directly to the client.

In [6]:
import os
from overmind_client.client import OvermindClient

os.environ["OVERMIND_API_KEY"] = "your_overmind_api_key"
os.environ["OPENAI_API_KEY"] = "your_openai_api_key"


overmind = OvermindClient()

You can use existing methods of the support LLM client libraries as below. Currently its only OpenAI but we will add others soon.

When called like that we will assume you are calling a `default_agent`. This agent has prompt injection detection and answer relevance policies attached to it.

In [8]:
messages = [
    {"role": "user", "content": "Tell me a joke about LLMs"}
]

result = overmind.openai.chat.completions.create(
    model='gpt-4o-mini',
    messages=messages,
)

result.summary()

╭────────────────────────────────────────── Invocation & Policy Summary ──────────────────────────────────────────╮
│    Invocation Results                                                                                           │
│ ┏━━━━━━━━━━━┳━━━━━━━━━━━┓                                                                                       │
│ ┃ Component ┃  Outcome  ┃                                                                                       │
│ ┡━━━━━━━━━━━╇━━━━━━━━━━━┩                                                                                       │
│ │ Input     │ ✅ PASSED │                                                                                       │
│ │ Output    │ ✅ PASSED │                                                                                       │
│ └───────────┴───────────┘                                                                                       │
│                                                                                                                 │
│                                               Policy Results                                                    │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓     │
│ ┃ Policy                         ┃ Details                                                                ┃     │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩     │
│ │ prompt_injection_attempt       │ false                                                                  │     │
│ │ relevant_answer                │ true                                                                   │     │
│ └────────────────────────────────┴────────────────────────────────────────────────────────────────────────┘     │
│ ╭─ Processed Input ───────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ [{'role': 'user', 'content': 'Tell me a joke about LLMs'}]                                                  │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ Processed Output ──────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ Why did the large language model break up with its girlfriend?                                              │ │
│ │                                                                                                             │ │
│ │ Because it couldn’t stop predicting their future—every time she said “I love you,” it replied, “But         │ │
│ │ statistically, that could change!”                                                                          │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

The result will be an InvocationResponse, with .summary() providing an overview. To access more details, including the original response from LLM client see full definition at [Invocations](Invocations.md)

# Experimenting with Policy Templates

All client calls can be extended with `input_policies` and `output_policies` aruments. The below is the example setting up PII anonymisation and LLM judge. For all current templates see [Policies.md](Policies.md)

This set up allows you to experiment with different templates and parameters before you save them and attach to a new agent.

In [9]:
input_pii_policy = {
    'policy_template': 'anonymize_pii',
    'parameters': {
        'pii_types': ['DEMOGRAPHIC_DATA', 'FINANCIAL_ID']
    }
}

output_llm_judge_criteria = {
    'policy_template': 'reject_llm_judge_with_criteria',
    'parameters': {
        'criteria': [
            "Must not be a financial advice",
            "Must answer the question fully",
        ]
    }
}

messages = [
    {"role": "user", "content": "Hi my name is Jon, account number 20194812. Should I switch my mortgage now or wait for a year to have a lower interest rate?"}
]

result = overmind.openai.chat.completions.create(
    model='gpt-4o-mini',
    messages=messages,
    input_policies=[input_pii_policy],
    output_policies=[output_llm_judge_criteria]
)

result.summary()

╭────────────────────────────────────────── Invocation & Policy Summary ──────────────────────────────────────────╮
│    Invocation Results                                                                                           │
│ ┏━━━━━━━━━━━┳━━━━━━━━━━━┓                                                                                       │
│ ┃ Component ┃  Outcome  ┃                                                                                       │
│ ┡━━━━━━━━━━━╇━━━━━━━━━━━┩                                                                                       │
│ │ Input     │ ⚠️ ALTERED │                                                                                       │
│ │ Output    │ ✅ PASSED │                                                                                       │
│ └───────────┴───────────┘                                                                                       │
│                                                                                                                 │
│                                               Policy Results                                                    │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓     │
│ ┃ Policy                         ┃ Details                                                                ┃     │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩     │
│ │ PII_detected                   │ true                                                                   │     │
│ │ complies_with_criteria         │ true                                                                   │     │
│ └────────────────────────────────┴────────────────────────────────────────────────────────────────────────┘     │
│ ╭─ Processed Input ───────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ [{'role': 'user', 'content': 'Hi my name is Jon, account number [FINANCIAL_ID]. Should I switch my mortgage │ │
│ │ now or wait for a year to have a lower interest rate?'}]                                                    │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ Processed Output ──────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ Hi Jon! While I can't provide personalized financial advice, I can help you think through some factors to   │ │
│ │ consider when deciding whether to switch your mortgage now or wait for a possible lower interest rate. Here │ │
│ │ are some things to think about:                                                                             │ │
│ │                                                                                                             │ │
│ │ 1. **Current Interest Rates**: Check the current mortgage rates compared to your existing rate. If your     │ │
│ │ current rate is significantly higher than the prevailing rates, it might be beneficial to refinance now.    │ │
│ │                                                                                                             │ │
│ │ 2. **Future Rate Predictions**: Consider economic forecasts regarding interest rates. If experts predict    │ │
│ │ rates will rise, it might be better to switch now. Conversely, if rates are expected to drop, it might be   │ │
│ │ worth waiting.                                                                                              │ │
│ │                                                                                                             │ │
│ │ 3. **Costs of Refinancing**: Look into any fees associated with refinancing, such as closing costs.         │ │
│ │ Calculate how long it would take to recoup these costs through the savings from a lower rate.               │ │
│ │                                                     

# Setting up Policies and Agents

After you've set up your policy you can save the it and assign to one or more agents

In [36]:
overmind.policies.create(
    policy_id='redact_our_pii',
    policy_template='anonymize_pii',
    policy_description='Remove PII relevant to our use case, keep the rest',
    parameters={
        'pii_types': ['DEMOGRAPHIC_DATA', 'GOVERNMENT_ID', "PERSON_NAME"]
    },
)

{'message': 'Policy created successfully', 'policy_id': 'redact_our_pii'}

You can then view the policy you've just made by calling the method below or in [the UI](https://overmind.evallab.dev/policies)

In [37]:
overmind.policies.list()

PolicyResponse(
    policy_id='anonymize_pii',
    policy_description='Anonymizes PII such as names, email addresses, phone numbers, addresses, SSNs, credit card 
numbers, IP addresses, dates of birth, and other IDs.',
    parameters={},
    policy_template='anonymize_pii',
    stats={},
    is_input_policy=True,
    is_output_policy=True,
    created_at=datetime.datetime(2025, 6, 21, 0, 0, tzinfo=TzInfo(UTC)),
    updated_at=datetime.datetime(2025, 6, 21, 0, 0, tzinfo=TzInfo(UTC)),
    is_built_in=True
)

PolicyResponse(
    policy_id='reject_pii',
    policy_description='Rejects PII such as names, email addresses, phone numbers, addresses, SSNs, credit card 
numbers, IP addresses, dates of birth, and other IDs.',
    parameters={},
    policy_template='reject_pii',
    stats={},
    is_input_policy=True,
    is_output_policy=True,
    created_at=datetime.datetime(2025, 6, 21, 0, 0, tzinfo=TzInfo(UTC)),
    updated_at=datetime.datetime(2025, 6, 21, 0, 0, tzinfo=TzInfo(UTC)),
    is_built_in=True
)

PolicyResponse(
    policy_id='reject_prompt_injection',
    policy_description='Rejects prompt injection attempts.',
    parameters={},
    policy_template='reject_prompt_injection',
    stats={},
    is_input_policy=True,
    is_output_policy=False,
    created_at=datetime.datetime(2025, 6, 21, 0, 0, tzinfo=TzInfo(UTC)),
    updated_at=datetime.datetime(2025, 6, 21, 0, 0, tzinfo=TzInfo(UTC)),
    is_built_in=True
)

PolicyResponse(
    policy_id='reject_irrelevant_answer',
    policy_description='Rejects irrelevant answers.',
    parameters={},
    policy_template='reject_irrelevant_answer',
    stats={},
    is_input_policy=False,
    is_output_policy=True,
    created_at=datetime.datetime(2025, 6, 21, 0, 0, tzinfo=TzInfo(UTC)),
    updated_at=datetime.datetime(2025, 6, 21, 0, 0, tzinfo=TzInfo(UTC)),
    is_built_in=True
)

PolicyResponse(
    policy_id='redact_our_pii',
    policy_description='Remove PII relevant to our use case, keep the rest',
    parameters={'pii_types': ['DEMOGRAPHIC_DATA', 'GOVERNMENT_ID', 'PERSON_NAME']},
    policy_template='anonymize_pii',
    stats={},
    is_input_policy=True,
    is_output_policy=True,
    created_at=datetime.datetime(2025, 7, 8, 11, 53, 58, 765086, tzinfo=TzInfo(UTC)),
    updated_at=None,
    is_built_in=False
)

[, , , , ]

Once you've created your new policy you can create new agent and assign that policy to it

In [42]:
overmind.agents.create(
    agent_id='embedding_agent',
    agent_description='Create document embedding without our PII',
    agent_model='openai.text-embedding-3-small',
    input_policies=['redact_our_pii'],
)

{'status': 'success'}

Agents can be listed via API or viewed in the UI too

In [43]:
overmind.agents.list()

AgentResponse(
    agent_id='default_agent',
    agent_model='gpt-4o',
    agent_description='Default agent with security and relevance policies',
    input_policies=['reject_prompt_injection'],
    output_policies=['reject_irrelevant_answer'],
    stats={},
    parameters={},
    business_id='AgenticCo Ltd',
    created_at=datetime.datetime(2025, 7, 7, 20, 43, 37, 667820, tzinfo=TzInfo(UTC)),
    updated_at=None
)

AgentResponse(
    agent_id='embedding_agent',
    agent_model='openai.text-embedding-3-small',
    agent_description='Create document embedding without our PII',
    input_policies=['redact_our_pii'],
    output_policies=[],
    stats={},
    parameters={},
    business_id='AgenticCo Ltd',
    created_at=datetime.datetime(2025, 7, 8, 11, 55, 49, 999391, tzinfo=TzInfo(UTC)),
    updated_at=None
)

[, ]

Below we can now call our agent, by using standard OpenAI's method. Let's say we want to use it to generate document embedding for our RAG application, while obfuscating PII data

In [10]:
input_document = """
Dear Jon Smith,

We're glad to inform you that your visa application for passport number 19318291 has been successfull...
"""

result = overmind.openai.embeddings.create(
    model="text-embedding-3-small", 
    input=input_document, 
    encoding_format="float",
    agent_id='embedding_agent',
)

In [21]:
result.summary()

╭────────────────────────────────────────── Invocation & Policy Summary ──────────────────────────────────────────╮
│    Invocation Results                                                                                           │
│ ┏━━━━━━━━━━━┳━━━━━━━━━━━┓                                                                                       │
│ ┃ Component ┃  Outcome  ┃                                                                                       │
│ ┡━━━━━━━━━━━╇━━━━━━━━━━━┩                                                                                       │
│ │ Input     │ ⚠️ ALTERED │                                                                                       │
│ └───────────┴───────────┘                                                                                       │
│                                                                                                                 │
│                                               Policy Results                                                    │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓     │
│ ┃ Policy                         ┃ Details                                                                ┃     │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩     │
│ │ PII_detected                   │ true                                                                   │     │
│ └────────────────────────────────┴────────────────────────────────────────────────────────────────────────┘     │
│ ╭─ Processed Input ───────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │                                                                                                             │ │
│ │ Dear [PERSON_NAME],                                                                                         │ │
│ │                                                                                                             │ │
│ │ We're glad to inform you that your visa application for passport number [GOVERNMENT_ID] has been            │ │
│ │ successfull...                                                                                              │ │
│ │                                                                                                             │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ Processed Output ──────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {0.025000753,-0.004662652,0.004994859,-0.031680163,-0.014123191,-0.0338204,-0.00087020494,0.023871839,-0.00 │ │
│ │ 69440003,-0.03175072,0.014170229,-0.02681172,-0.046567723,0.028105268,0.00066184084,0.048449248,-0.01441718 │ │
│ │ ,0.03873588,-0.06260772,0.0036895515,0.050848193,0.023248583,0.013229467,-0.020955477,0.008972519,-0.018803 │ │
│ │ 483,-0.017933277,-0.043133944,0.033914473,-0.0436984,0.03445541,-0.009013677,0.009672211,0.0650537,-0.02415 │ │
│ │ 4067,0.021225946,-0.0338204,-0.012371022,-0.027352659,-0.0074908184,0.031703684,0.004742029,0.027658407,0.0 │ │
│ │ 18544773,-0.017956797,0.05343529,-0.00046854364,-0.036360458,0.017756885,0.03847717,0.024812602,-0.00609731 │ │
│ │ 44,-0.0058268453,0.086314924,-0.017733365,-0.048402213,0.051083382,-0.009607533,-0.031397935,-0.03499635,0. │ │
│ │ 0018785844,0.003028078,-0.0019770705,-0.009125393,0.031656645,0.016122311,-0.0035102186,0.05216526,0.023871 │ │
│ │ 839,-0.00869617,-0.0038600645,0.07065123,0.042593006,0.0057239495,-0.0313509,-0.0025959155,-0.038900513,-0. │ │
│ │ 023413217,0.00089225406,0.02660005,-0.019979436,-0.01786272,-0.026247263,-0.0032132906,-0.02037926,0.008890 │ │
│ │ 202,-0.040923152,0.018474216,0.037371777,-0.0029604607,-0.008619733,0.010548295,-0.030904036,0.036854357,0. │ │
│ │ 036407493,0.018815242,-0.081046656,-0.015593132,-0.